In [2]:
import pandas as pd
import os
import datetime as dt

In [3]:
df_ = pd.read_excel('./online_retail_ii.xlsx',sheet_name='Year 2010-2011')

In [4]:
df = df_.copy()
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [5]:
#PREPROCESSING
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [6]:
df['Total_Price'] = df['Quantity']*df['Price']

In [8]:
df.dropna(inplace=True)
df=df[-df['Invoice'].str.contains('C',na=False)]
#Values begins with 'C' means returned transactions.

In [9]:
#We will use this date to create Recency column by finding the difference between today date 
# and transaction date and we'll see how long it has been since the last transaction of customer
today_date = dt.datetime(2011,12,11)

In [10]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Total_Price'],
      dtype='object')

In [11]:
#Calculation of RFM metrics
rfm = df.groupby('Customer ID').agg({'InvoiceDate':lambda InvoiceDate:(today_date-InvoiceDate.max()).days,
                                     'Invoice':lambda Invoice:Invoice.nunique(),
                                     'Total_Price':lambda Total_Price: Total_Price.sum()})

In [12]:
rfm=rfm.rename(columns={'InvoiceDate':'Recency','Invoice':'Frequency','Total_Price':'Monetary Value'})

In [13]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4339.0,93.041484,100.007757,1.0,18.000,51.00,142.50,374.00
Frequency,4339.0,4.271952,7.705493,1.0,1.000,2.00,5.00,210.00
Monetary Value,4339.0,2053.797166,8988.247836,0.0,307.245,674.45,1661.64,280206.02


In [14]:
#We take values on Monetary Value column which are bigger than 0 because it includes zeros in it. 
#It is meaningless and obviously an outlier because the sum of all the transactions can't be equal to 0. 
rfm = rfm[rfm['Monetary Value']>0]

In [15]:
rfm

,Recency,Frequency,Monetary Value
Customer ID,,,
12346.0,326,1,77183.60
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,8,2,178.05


In [16]:
#RFM SCORING

In [17]:
rfm['recency_score'] = pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])
rfm['frequency_score']= pd.qcut(rfm['Frequency'].rank(method='first'),5,labels=[1,2,3,4,5])
rfm['monetary _score']= pd.qcut(rfm['Monetary Value'],5,labels=[1,2,3,4,5])
rfm['RFM SCORE but without M'] =(rfm['recency_score'].astype(str) +
                                 rfm['frequency_score'].astype(str)
)

In [18]:
rfm

,Recency,Frequency,Monetary Value,recency_score,frequency_score,monetary _score,RFM SCORE but without M
Customer ID,,,,,,,
12346.0,326,1,77183.60,1,1,5,11
12347.0,3,7,4310.00,5,5,5,55
12348.0,76,4,1797.24,2,4,4,24
12349.0,19,1,1757.55,4,1,4,41
12350.0,311,1,334.40,1,1,2,11
...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,2,1,12
18281.0,181,1,80.82,1,2,1,12
18282.0,8,2,178.05,5,3,1,53


In [19]:
#List perfect cusomers
rfm[rfm['RFM SCORE but without M']=='55']

,Recency,Frequency,Monetary Value,recency_score,frequency_score,monetary _score,RFM SCORE but without M
Customer ID,,,,,,,
12347.0,3,7,4310.00,5,5,5,55
12362.0,4,10,5226.23,5,5,5,55
12417.0,4,9,3649.10,5,5,5,55
12423.0,1,8,1859.31,5,5,4,55
12433.0,1,7,13375.87,5,5,5,55
...,...,...,...,...,...,...,...
18230.0,10,7,2810.20,5,5,5,55
18241.0,10,17,2073.09,5,5,5,55
18245.0,8,7,2567.06,5,5,5,55


In [20]:
#RFM Segmentation
seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

In [21]:
rfm['segment'] = rfm['RFM SCORE but without M'].replace(seg_map,regex=True)

In [22]:
rfm

,Recency,Frequency,Monetary Value,recency_score,frequency_score,monetary _score,RFM SCORE but without M,segment
Customer ID,,,,,,,,
12346.0,326,1,77183.60,1,1,5,11,hibernating
12347.0,3,7,4310.00,5,5,5,55,champions
12348.0,76,4,1797.24,2,4,4,24,at_risk
12349.0,19,1,1757.55,4,1,4,41,promising
12350.0,311,1,334.40,1,1,2,11,hibernating
...,...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,2,1,12,hibernating
18281.0,181,1,80.82,1,2,1,12,hibernating
18282.0,8,2,178.05,5,3,1,53,potential_loyalists


In [23]:
#Count of customers at each segment
rfm.groupby('segment').size()

segment
about_to_sleep          352
at_risk                 593
cant_loose               63
champions               633
hibernating            1071
loyal_customers         819
need_attention          187
new_customers            42
potential_loyalists     484
promising                94
dtype: int64